In [1]:
!pip -q install crewai langchain-groq duckduckgo-search

DEPRECATION: Loading egg at c:\program files\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [2]:
!pip show crewai serpapi

Name: crewai
Version: 0.30.11
Summary: Cutting-edge framework for orchestrating role-playing, autonomous AI agents. By fostering collaborative intelligence, CrewAI empowers agents to work together seamlessly, tackling complex tasks.
Home-page: 
Author: Joao Moura
Author-email: joao@crewai.com
License: 
Location: C:\Users\shiv\AppData\Roaming\Python\Python311\site-packages
Requires: appdirs, click, embedchain, instructor, langchain, openai, opentelemetry-api, opentelemetry-exporter-otlp-proto-http, opentelemetry-sdk, pydantic, python-dotenv, regex
Required-by: 
---
Name: serpapi
Version: 0.1.5
Summary: The official Python client for SerpApi.com.
Home-page: https://github.com/serpapi/serpapi-python
Author: SerpApi.com
Author-email: kenneth@serpapi.com
License: MIT
Location: C:\Users\shiv\AppData\Roaming\Python\Python311\site-packages
Requires: requests
Required-by: 


DEPRECATION: Loading egg at c:\program files\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [1]:
from dotenv import load_dotenv
from crewai import Crew, Agent, Task, Process
from langchain_groq import ChatGroq
from langchain_community.tools import DuckDuckGoSearchRun
import json
import os
import requests

load_dotenv()

True

## Using Groq Model

In [2]:
GROQ_LLM = ChatGroq(model="llama3-70b-8192")

In [3]:
def fetch_news(query, api_key=os.getenv("SERP_API_KEY")):
    params = {
        "engine": "google",
        "q": query,
        "tbm": "nws",
        "api_key": api_key
    }
    response = requests.get("https://google.serper.dev/search", params=params)
    return response.json()

## AI Agents

In [4]:
# Define the agents
class NewsAgents():
    def make_search_agent(self):
        return Agent(
            role='News Search Agent',
            goal="""Search for news articles using a given query.""",
            backstory="""You are skilled at finding relevant news articles on the internet.""",
            llm=GROQ_LLM,
            verbose=True,
            allow_delegation=False,
            max_iter=5,
            memory=True,
            tools=[DuckDuckGoSearchRun()],
        )

    def make_summary_agent(self):
        return Agent(
            role='Article Summary Agent',
            goal="""Summarize the news articles.""",
            backstory="""You excel at creating concise summaries of news articles.""",
            llm=GROQ_LLM,
            verbose=True,
            allow_delegation=False,
            max_iter=5,
            memory=True,
        )

    def make_keyword_agent(self):
        return Agent(
            role='Keyword Extraction Agent',
            goal="""Extract key words from the news articles.""",
            backstory="""You are adept at identifying key words from text.""",
            llm=GROQ_LLM,
            verbose=True,
            allow_delegation=False,
            max_iter=5,
            memory=True,
        )

## Making Tasks

In [5]:
# Define the tasks
class NewsTasks():
    def search_news(self, query):
        return Task(
            description=f"""Search for news articles with the query: {query}.""",
            expected_output="""A list of news articles with details like date, region, source, article summary, and keywords.""",
            output_file=f"news_articles.txt",
            agent=search_agent
        )

    def summarize_article(self, article):
        return Task(
            description=f"""Summarize the following news article: {article}.""",
            expected_output="""A summary of the article.""",
            context=[search_news],
            output_file=f"article_summary.txt",
            agent=summary_agent
        )

    def extract_keywords(self, article):
        return Task(
            description=f"""Extract key words from the following news article: {article}.""",
            expected_output="""A list of key words from the article.""",
            context=[search_news],
            output_file=f"article_keywords.txt",
            agent=keyword_agent
        )

In [6]:
# Example query
QUERY = "latest news in indial electrical industry"

## Instantiate agents and tasks

In [7]:
agents = NewsAgents()
tasks = NewsTasks()

search_agent = agents.make_search_agent()
summary_agent = agents.make_summary_agent()
keyword_agent = agents.make_keyword_agent()

search_news = tasks.search_news(QUERY)


# Formating news Articals

In [8]:
def process_news_articles(news_data):
    articles = news_data.get('news_results', [])
    formatted_articles = []
    for article in articles:
        date = article.get('date')
        region = article.get('region', 'N/A')
        source = article.get('source')
        title = article.get('title')
        summary_task = tasks.summarize_article(title)
        keywords_task = tasks.extract_keywords(title)

        summary = summary_task.agent.complete_task(title)
        keywords = keywords_task.agent.complete_task(title)

        formatted_articles.append({
            "date": date,
            "region": region,
            "source": source,
            "article_summary": summary,
            "keywords": keywords
        })

    return formatted_articles


In [9]:
# Fetch news articles 
news_data = fetch_news(QUERY)

In [10]:
formatted_articles = process_news_articles(news_data)

## Print the results

In [11]:
for article in formatted_articles:
    print(f"Date: {article['date']}")
    print(f"Region: {article['region']}")
    print(f"Source: {article['source']}")
    print(f"Article Summary: {article['article_summary']}")
    print(f"Keywords: {article['keywords']}")
    print("\n")

# with open('formatted_articles.json', 'w') as f:
#     json.dump(formatted_articles, f, indent=4)

In [12]:
def print_agent_output(agent_output, agent_name):
    if isinstance(agent_output, str):
        print(f"{agent_name}: {agent_output}")
    elif isinstance(agent_output, list):
        for item in agent_output:
            print(f"{agent_name}: {item}")
    else:
        print(f"{agent_name}: Unknown output format")

## crew-AI

In [13]:
crew = Crew(
    agents=[search_agent, summary_agent, keyword_agent],
    tasks=[search_news],
    verbose=2,
    process=Process.sequential,
    full_output=True,
    share_crew=False,
    step_callback=lambda x: print_agent_output(x,"MasterCrew Agent")
)

In [14]:
results = crew.kickoff()

print("Crew Work Results:")
print(results)

print(crew.usage_metrics)

 [DEBUG]: == Working Agent: News Search Agent
 [INFO]: == Starting Task: Search for news articles with the query: latest news in indial electrical industry.


> Entering new CrewAgentExecutor chain...
I need to find relevant news articles related to the latest news in the Indian electrical industry.

Action: duckduckgo_search
Action Input: {"q": "latest news in Indian electrical industry"} 

I encountered an error while trying to use the tool. This was the error: DuckDuckGoSearchRun._run() got an unexpected keyword argument 'q'.
 Tool duckduckgo_search accepts these inputs: A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.

MasterCrew Agent: (AgentAction(tool='duckduckgo_search', tool_input='{"q": "latest news in Indian electrical industry"}', log='I need to find relevant news articles related to the latest news in the Indian electrical industry.\n\nAction: duckduckgo_search\nAction Input: {"q": "lates